# Equal Weighted S&P Index

The goal of this section is to create a python script which takes portfolio size as an input and provides the number of shares of each S&P constituent to be purchased to create an equal weighted S&P index fund.

In [ ]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing the list of stocks

The constituents of S&P index keeps on changing and thus needs to be updated overtime. We have used a list provided in the reference.

In [68]:
stocks = pd.read_csv('sp_500_stocks.csv')

# Acquiring an API Token

IEX Cloud API is the data provider that we will be using in thsi script. We need to import IEX Cloud API token to be used in this script. 

We will be using a free, sand box API token for this course and thus the data generated using this token will not match with the actual data. For getting actual data, we need to go for a paid plan. 

In [69]:
from secrets import IEX_CLOUD_API_TOKEN

In [70]:
base_url = "https://sandbox.iexapis.com/stable"
symbol = "AAPL"
api_url = f'{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

In [71]:
request = requests.get(api_url)
print(type(request))
print(request.status_code == 200)

<class 'requests.models.Response'>
True


In [72]:
data = request.json()
print(data.keys())
print(data)

dict_keys(['symbol', 'companyName', 'primaryExchange', 'calculationPrice', 'open', 'openTime', 'openSource', 'close', 'closeTime', 'closeSource', 'high', 'highTime', 'highSource', 'low', 'lowTime', 'lowSource', 'latestPrice', 'latestSource', 'latestTime', 'latestUpdate', 'latestVolume', 'iexRealtimePrice', 'iexRealtimeSize', 'iexLastUpdated', 'delayedPrice', 'delayedPriceTime', 'oddLotDelayedPrice', 'oddLotDelayedPriceTime', 'extendedPrice', 'extendedChange', 'extendedChangePercent', 'extendedPriceTime', 'previousClose', 'previousVolume', 'change', 'changePercent', 'volume', 'iexMarketPercent', 'iexVolume', 'avgTotalVolume', 'iexBidPrice', 'iexBidSize', 'iexAskPrice', 'iexAskSize', 'iexOpen', 'iexOpenTime', 'iexClose', 'iexCloseTime', 'marketCap', 'peRatio', 'week52High', 'week52Low', 'ytdChange', 'lastTradeTime', 'isUSMarketOpen'])
{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'QKM )SAOASDA( C/LL SREBGTNAGNTEEL', 'calculationPrice': 'close', 'open': 127.56, 'openTi

In [73]:
price = data["latestPrice"]
market_cap = data["marketCap"]
print(price)
print(market_cap)

128.21
2178551628171


In [74]:
my_columns = ['Tickers', 'Stock Price', 'Market Capitalization', 'Number of Stocks to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Stocks to Buy


In [75]:
final_dataframe = final_dataframe.append(
    pd.Series([
        symbol,
        data["latestPrice"],
        data["marketCap"],
        'N/A'
    ],
        index = my_columns),
    ignore_index = True)
    
final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Stocks to Buy
0,AAPL,128.21,2178551628171,N/A


# Loop through tickers in S&P stock list

Here we will loop through all the tickers in the S&P stock list using a for loop. We can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.We will pull all the price and market cap data for S&P stocks and save all these data in a panda data frame. Since the call will be serial one and so it will take time to get the result.

In [76]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol in stocks['Ticker'][:10]:
    api_url = f'{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                    pd.Series([
                                        symbol,
                                        data["latestPrice"],
                                        data["marketCap"],
                                        'N/A'
                                        ],
                                        index = my_columns),
                                    ignore_index = True)
final_dataframe    

,Tickers,Stock Price,Market Capitalization,Number of Stocks to Buy
0,A,117.19,36490232282,N/A
1,AAL,16.50,10027788314,N/A
2,AAP,152.57,10439774260,N/A
3,AAPL,124.20,2112593200432,N/A
4,ABBV,110.22,189495015746,N/A
5,ABC,105.38,21134028436,N/A
6,ABMD,275.14,12464915645,N/A
7,ABT,108.60,197751821034,N/A
8,ACN,257.64,169017543864,N/A
9,ADBE,496.00,238083041240,N/A


# Using batch API call to improve performance

For this we will divide the calls into chunks with each chunk containing 100 stocks. 

In [77]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

Here we divide all the stocks into chunks where each chunk is a list of 100 stocks.

In [78]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_groups[i])
#     print(symbol_strings[i])
    
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
    print(len(symbol_strings[i]))


print(len(stocks['Ticker']))   
print(len(symbol_groups[0]))
print(len(symbol_groups[1]))
print(len(symbol_groups[2]))
print(len(symbol_groups[3]))
print(len(symbol_groups[4]))
print(len(symbol_groups[5]))

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
421
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
414
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

Now we will make batch api call for each chunk. In this case there will be only 6 call for getting the response for all the stocks. Since rresponse time for batch call is less so it will take less time to get response in batch call.

In [89]:
final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'{base_url}/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        #print(symbol)
        final_dataframe = final_dataframe.append(
                                    pd.Series([
                                        symbol,
                                        data[symbol]['quote']['latestPrice'],
                                        data[symbol]['quote']['marketCap'],
                                        'N/A'
                                        ],
                                        index = my_columns),
                                    ignore_index = True)
final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Stocks to Buy
0,A,118.070,36018953515,N/A
1,AAL,17.100,10051267580,N/A
2,AAP,154.860,10819428048,N/A
3,AAPL,125.690,2119754820719,N/A
4,ABBV,109.580,194093736441,N/A
...,...,...,...,...
500,YUM,111.130,32005818867,N/A
501,ZBH,156.010,31199766180,N/A
502,ZBRA,379.880,21120842383,N/A
503,ZION,44.283,7570452657,N/A


# Find the number of shares to buy

Now we will calculate the number of share to buy in an equal weighted schenario. 

In [90]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [93]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Stocks to Buy'] = math.floor(position_size/final_dataframe['Stock Price'][i])

final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Stocks to Buy
0,A,118.070,36018953515,16
1,AAL,17.100,10051267580,115
2,AAP,154.860,10819428048,12
3,AAPL,125.690,2119754820719,15
4,ABBV,109.580,194093736441,18
...,...,...,...,...
500,YUM,111.130,32005818867,17
501,ZBH,156.010,31199766180,12
502,ZBRA,379.880,21120842383,5
503,ZION,44.283,7570452657,44


In [94]:
final_dataframe.to_csv("Recommended Trades.csv", index = None)

### Reference: 
https://www.youtube.com/watch?v=xfzGZB4HhEE&ab_channel=freeCodeCamp.org